<p style="text-align:center"><img src=../images/header.png/></p>

<p style="text-align:center"><img src="https://camo.githubusercontent.com/4a300a0747dc7b5c69a83cc94ff2e2b5ad1f0f937ed3d142b171c0d4a6f8e164/68747470733a2f2f7777772e67652e636f6d2f72656e657761626c65656e657267792f73697465732f64656661756c742f66696c65732f323032302d30312f6f6e73686f72652d6865726f352e6a7067"/></p>


<h1 align="center">This script transfers the preprocessed data to the CE4IoT WindIO instance.</h1>

In [1]:
import ast
import pandas as pd
import requests
import json

#### Please fill the needed infromation.

In [4]:
pre_path = "./preprocessed/measurement_windio_msb-0004-a_2021-10-21.csv"
json_path = "./measurement_windio_msb-0004-a_2021-10-21.json"
instance_url = ""
user_name = ''
password = ''
asset_name = "AS-000003-0003"
time_series_name = "measurement_windio_msb-0004-a_2021-10-21"

#### Execute this cell to transfer your data to the CE4IoT instance.

In [5]:
df = pd.read_csv(pre_path, parse_dates=["date_time"])
df.rename(columns = {'date_time':'time'}, inplace=True)
df["time"] = pd.to_datetime(df["time"], unit='s')
df = df.to_json(json_path, orient = 'records')

with open(json_path) as json_file:
    json_data = json.load(json_file)
    
def get_csrf_token(session):
    csrf = session.cookies.get('CSRFToken')
    if csrf is None:
        session.get(instance_url + "/server/sessioninfo")
        csrf = session.cookies.get('CSRFToken')
    return csrf

with requests.Session() as session:
    session.auth = (user_name, password)
    headers = {
        "X-Csrf-Token": get_csrf_token(session)
    }
    resp = session.post(instance_url + "/iot_api/v1/assets/" + asset_name + "/time_series/" + time_series_name + "/data", json=json_data[], headers=headers)
    print(resp)
    resp = session.get(instance_url + '/server/__quit__')